# Compress Model

In [6]:
import torch.nn as nn
import torch.nn.functional as F
from sparsecomp import compress_NN_models
import torch
from conversion import save_compressed_model
import torchvision.transforms as transforms

import os
os.environ["CUDA_VISIBLE_DEVICES"]="GPU-949091da-1455-7238-7234-08221ff71c62"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [7]:
def print_full_model(model):
    assert isinstance(model, nn.Module), "The model is not a subclass of torch.nn.Module"
    kb = 1000
    model_size = 0
    params = 0
    for name, param in model.named_parameters():
        layer_size = param.nelement() * param.element_size()
        model_size += layer_size
        # print(name,"\t", param.nelement(), "\t", param.element_size(),"\t", layer_size / kb, "KB")

    for name, buffer in model.named_buffers():
        layer_size = buffer.nelement() * buffer.element_size()
        model_size += layer_size
        # print(name,"\t", layer_size / kb, "KB")
    # print("Model Size:", model_size / kb, "KB")

    params = sum(p.numel() for p in model.parameters())
    # print("Model Params:", params)

    return (model_size / kb), params

In [8]:
class Baseline_compressed(nn.Module):
    def __init__(self):
        super(Baseline_compressed, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, padding="valid")
        self.pool1 = nn.MaxPool2d(kernel_size=(2, 2))
        self.conv2 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding="valid")
        self.pool2 = nn.MaxPool2d(kernel_size=(2, 2))
        self.conv3 = nn.Conv2d(in_channels=128, out_channels=64, kernel_size=3, padding="valid")
        self.pool3 = nn.MaxPool2d(kernel_size=(2, 2))
        
        self.flatten = nn.Flatten()
        
        self.fc1 = nn.Sequential(
            nn.Linear(in_features=256, out_features=25, bias=False),
            nn.Linear(in_features=25, out_features=256)
        )
        self.fc2 = nn.Linear(in_features=256, out_features=64)
        self.fc3 = nn.Linear(in_features=64, out_features=10)
        
    def forward(self, x):
        exit_outputs = []
        x = self.conv1(x)
        x = F.relu(x)
        x = self.pool1(x)
        exit_outputs.append(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = self.pool2(x)
        exit_outputs.append(x)
        x = self.conv3(x)
        x = F.relu(x)
        x = self.pool3(x)
        exit_outputs.append(x)
        
        x = self.flatten(x)
        
        x = self.fc1[0](x)
        x = self.fc1[1](x)
        x = self.fc2(x)
        x = self.fc3(x)
        
        return x, exit_outputs


In [9]:
class Baseline(nn.Module):
    def __init__(self):
        super(Baseline, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, padding="valid")
        self.pool1 = nn.MaxPool2d(kernel_size=(2, 2))
        self.conv2 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding="valid")
        self.pool2 = nn.MaxPool2d(kernel_size=(2, 2))
        self.conv3 = nn.Conv2d(in_channels=128, out_channels=64, kernel_size=3, padding="valid")
        self.pool3 = nn.MaxPool2d(kernel_size=(2, 2))
        
        self.flatten = nn.Flatten()
        
        self.fc1 = nn.Linear(in_features=256, out_features=256)
        self.fc2 = nn.Linear(in_features=256, out_features=64)
        self.fc3 = nn.Linear(in_features=64, out_features=10)
        
    def forward(self, x):
        exit_outputs = []
        x = self.conv1(x)
        x = F.relu(x)
        x = self.pool1(x)
        exit_outputs.append(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = self.pool2(x)
        exit_outputs.append(x)
        x = self.conv3(x)
        x = F.relu(x)
        x = self.pool3(x)
        exit_outputs.append(x)
        
        x = self.flatten(x)
        
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        
        return x, exit_outputs


In [10]:
#for S_earlyexit_model
class GeneralEEModel(nn.Module):
    def __init__(self):
        super(GeneralEEModel, self).__init__()
        self.pool_kernels = [
            (1, 6, 6), (1, 3, 3), (1, 1, 1)
        ]
        self.flatten = nn.Flatten()
        self.dropout = nn.Dropout(p=0.4)
        self.fc = nn.Linear(in_features=1024, out_features=10)
    
    def forward(self, x, inference=False):
        pooled_outs = []
        for layer, out in enumerate(x):
            pool_3d = nn.MaxPool3d(kernel_size=self.pool_kernels[layer])
            pooled_outs.append(pool_3d(x))
        x = torch.cat(pooled_outs, dim=1)
        x = self.flatten(x)
        x = self.dropout(x)
        scores = self.fc(x)
        return scores

In [11]:
import torch
from torchvision.datasets import CIFAR10
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader, random_split

# Define your model (assuming GeneralEEModel is defined elsewhere)
# model = GeneralEEModel().to(device)
model = Baseline().to(device)
model_size, params = print_full_model(model)
print("Baseline-S Model Size:", model_size, "KB")
print("Baseline-S Model Params:", params)


# Load CIFAR-10 dataset
dataset = CIFAR10(root='./data', download=True, transform=ToTensor())
test_dataset = CIFAR10(root='./data', train=False, transform=ToTensor())

# # Define data loaders
# batch_size = 128
# val_size = 5000
# train_size = len(dataset) - val_size
# train_ds, val_ds = random_split(dataset, [train_size, val_size])
# train_loader = DataLoader(train_ds, batch_size, shuffle=True, num_workers=4)
# val_loader = DataLoader(val_ds, batch_size * 2, num_workers=4)
# test_loader = DataLoader(test_dataset, batch_size * 2, num_workers=4)

# # Define other parameters
# target_size = 32  # Target size in KB
# num_epochs = 10
# learning_rate = 0.001
# criterion = torch.nn.CrossEntropyLoss()
# regularizerParam = 0.0
# compressionStep = 0.1
# initialCompressionStep = 0.1
# fastCompression = False
# modelName = "_32KB"
# device = "cpu"
# accuracyAware = True
# layersFactorization = True
# calculateInputs = None

# # Call the compress_NN_models function
# compress_NN_models(
#     model, target_size, train_loader, test_loader,
#     val_loader=val_loader, num_epochs=num_epochs, learning_rate=learning_rate,
#     criterion=criterion, regularizerParam=regularizerParam, compressionStep=compressionStep,
#     initialCompressionStep=initialCompressionStep, fastCompression=fastCompression,
#     modelName=modelName, device=device, accuracyAware=accuracyAware,
#     layersFactorization=layersFactorization, calculateInputs=calculateInputs
# )

Baseline-S Model Size: 929.32 KB
Baseline-S Model Params: 232330
Files already downloaded and verified


In [9]:
model = Baseline_compressed().to(device)
# print('Expected model keys: \n',model.state_dict().keys())  # Expected keys
# print('loaded model keys: \n',torch.load("compressed_model_186.h5").keys())  # Loaded keys
model.load_state_dict(torch.load("/home/mal/DScale/momin_flex_nns/freeml/FreeML/SparseComp/_32KB_30.h5", map_location='cpu'))
model_size, params = print_full_model(model)
print("Baseline-S Model Size:", model_size, "KB")
print("Baseline-S Model Params:", params)

# Load CIFAR-10 dataset
transform = transforms.Compose([
    transforms.ToTensor(),  # Convert to tensor
])

# Download dataset and get a single sample
single_sample, _ = dataset[0]  # Extract first sample (image, label)

# Add batch dimension
single_sample = single_sample.unsqueeze(0)  # Shape: (1, 3, 32, 32)

save_compressed_model(model, 'csr', input_data=single_sample, directory='_32KB_C_code')

Baseline-S Model Size: 718.376 KB
Baseline-S Model Params: 179594
Directory '_32KB_C_code' created
Directory '_32KB_C_code/headers' created
Values: 0 0

Iterating over the module:

Weight shape: torch.Size([64, 3, 3, 3])
Shape output of the module: torch.Size([1, 64, 30, 30])
Layer conv 1 Values: 1 1
Shape output of the module: torch.Size([1, 64, 15, 15])
Output Dimension: 4
Layer pooling 1 Values: 2 2
Weight shape: torch.Size([128, 64, 3, 3])
Shape output of the module: torch.Size([1, 128, 13, 13])
Layer conv 2 Values: 3 3
Shape output of the module: torch.Size([1, 128, 6, 6])
Output Dimension: 4
Layer pooling 2 Values: 4 4
Weight shape: torch.Size([64, 128, 3, 3])
Shape output of the module: torch.Size([1, 64, 4, 4])
Layer conv 3 Values: 5 5
Shape output of the module: torch.Size([1, 64, 2, 2])
Output Dimension: 4
Layer pooling 3 Values: 6 6
Shape output of the module: torch.Size([1, 256])
Layer flatten 1 Values: 7 7
Weight shape: torch.Size([25, 256])
Shape output of the module: tor

In [13]:
from torchsummary import summary
model = Baseline_compressed().to(device)
# print('Expected model keys: \n',model.state_dict().keys())  # Expected keys
# print('loaded model keys: \n',torch.load("compressed_model_186.h5").keys())  # Loaded keys
model.load_state_dict(torch.load("/home/mal/DScale/momin_flex_nns/freeml/FreeML/SparseComp/models/32KB/_32KB_30.h5", map_location='cpu'))
summary(model, (3, 32, 32))
print(100*'-','\n')
print("Actual active weights per layer\n")
print(100*'-','\n')
for name, layer in model.named_modules():
    if isinstance(layer, (torch.nn.Conv2d, torch.nn.Linear)):
        print(name, layer.weight.data.nonzero().size(0))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 30, 30]           1,792
         MaxPool2d-2           [-1, 64, 15, 15]               0
            Conv2d-3          [-1, 128, 13, 13]          73,856
         MaxPool2d-4            [-1, 128, 6, 6]               0
            Conv2d-5             [-1, 64, 4, 4]          73,792
         MaxPool2d-6             [-1, 64, 2, 2]               0
           Flatten-7                  [-1, 256]               0
            Linear-8                   [-1, 25]           6,400
            Linear-9                  [-1, 256]           6,656
           Linear-10                   [-1, 64]          16,448
           Linear-11                   [-1, 10]             650
Total params: 179,594
Trainable params: 179,594
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.01
Forward/